In [3]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from orderedset import OrderedSet
from copy import deepcopy
%matplotlib inline  

In [7]:
flatten = lambda l: [item for sublist in l for item in sublist]

datatypes = ['string', 'boolean', 'decimal', 'float', 'double', 'dateTime', 'time', 'date', 'gYearMonth', 'gYear',
             'gMonthDay', 'gDay', 'gMonth', 'hexBinary', 'base64Binary', 'anyURI', 'normalizedString', 'token', 'language', 
             'NMTOKEN', 'Name', 'NCName', 'integer', 'nonPositiveInteger', 'negativeInteger', 'long', 'int', 'short', 'byte',
             'nonNegativeInteger', 'unsignedLong', 'unsignedInt', 'unsignedShort', 'unsignedByte', 'positiveInteger']

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.parents_dict = {}
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        subclasses = self.get_subclasses()
        self.parents_dict = {}
        for (a,b,c,d) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [(b,a,c,d) for (a,b,c,d) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True):
        obj_props = [(prop, "Object Property") for prop in self.object_properties]
        data_props = [(prop, "Datatype Property") for prop in self.data_properties]
        props = obj_props + data_props
        all_triples = []
        for prop, prop_type in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop), prop_type) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop), prop_type))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True):
        return self.parse_triples(union_flag, subclass_of)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of", "Subclass"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
#                     print(self.extract_ID(el), "**", self.extract_ID(some_vals[0]), "**", self.extract_ID(prop[0]))
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                    except:
                        try:
                            if not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                            elif not self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                class_vals = self.get_child_node(some_vals[0], "owl:Class")
                                subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                        except Exception as e:
                            print (e)
                            continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of", "Subclass"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of", "Subclass") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
        
    def get_subclasses(self):
        subclasses = [(self.extract_ID(a), self.extract_ID(b), c, d) for (a,b,c,d) in self.subclasses]
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-2] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


In [11]:
USE_folder = "/home/vlead/USE"
alignment_folder = "AML-conference/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2'), doc.getElementsByTagName('measure')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource'), c.firstChild.nodeValue) for (a,b,c) in ls])
    return alignments

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)


reference_alignments = load_alignments(alignment_folder)

# ra_anatomy_coded = load_alignments("../Anatomy/Alignments/")
# ra_anatomy = []
# ont1 = Ontology("../Anatomy/Ontologies/mouse.owl")
# ont2 = Ontology("../Anatomy/Ontologies/human.owl")
# for elem in ra_anatomy_coded:
#     pre1, pre2 = elem[0].split("#")[0].split(".")[0].split("/")[-1], elem[1].split("#")[0].split(".")[0].split("/")[-1]
#     elem1, elem2 = elem[0].split("#")[-1], elem[1].split("#")[-1]
#     ra_anatomy.append(( pre1 + "#" + ont1.mapping_dict[elem1], pre2 + "#" + ont2.mapping_dict[elem2]))

gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]
# gt_mappings.extend(ra_anatomy)


ontologies_in_alignment = pickle.load(open("../data_generic.pkl", "rb"))[-1][:-1]
# ontologies_in_alignment += [["../Anatomy/Ontologies/human.owl", "../Anatomy/Ontologies/mouse.owl"]]


In [12]:
# Combinatorial mapping generation
ent_mappings = []
prop_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology("conference_ontologies/" + l[0] + ".owl")
    ont2 = Ontology("conference_ontologies/" + l[1] + ".owl")
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()

    ent_mapping = list(itertools.product(ent1, ent2))
    prop_mapping = list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    ent_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in ent_mapping])
    prop_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in prop_mapping])

aml_data_ent = {mapping: 0 for mapping in ent_mappings}
aml_data_prop = {mapping: 0 for mapping in prop_mappings}


s_ent = set(ent_mappings)
s_prop = set(prop_mappings)
for mapping in set(gt_mappings):
    if mapping[:2] in s_ent:
        aml_data_ent[tuple(mapping[:2])] = mapping[-1]
    elif mapping[:2] in s_prop:
        aml_data_prop[tuple(mapping[:2])] = mapping[-1]
    else:
        print (mapping)


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [9]:
# Abbrevation resolution preprocessing

abbreviations_dict = {}
final_dict = {}

for mapping in ent_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


left ('Member_PC', 'Program_Committee') PC Program_Committee
left ('Member_PC', 'Program_Chair') PC Program_Chair
left ('Member_PC', 'Program_Committee_member') PCM Program_Committee
left ('Chair_PC', 'Program_Committee') PC Program_Committee
left ('Chair_PC', 'Program_Chair') PC Program_Chair
left ('Chair_PC', 'Program_Committee_member') PCM Program_Committee
left ('Member_PC', 'Presenter_city') PC Presenter_city
left ('Chair_PC', 'Presenter_city') PC Presenter_city
left ('OC_Member', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Member', 'Organizing_Committee') OC Organizing_Committee
left ('OC_Chair', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Chair', 'Organizing_Committee') OC Organizing_Committee
left ('PC_Member', 'Program_Committee') PC Program_Committee
left ('PC_Member', 'Program_Chair') PC Program_Chair
left ('PC_Member', 'Program_Committee_member') PCM Program_Committee
left ('PC_Chair', 'Program_Committee') PC Program_Committee
lef

In [10]:

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]

# Resolving abbreviations to full forms
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    final_list = []
    # Lowering case except in abbreviations
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", word):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    concept = " ".join(final_list)
    inp_resolved.append(concept)


url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

inp_spellchecked = []
for concept in inp_resolved:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        final_list = []
        for word in concept.split(" "):
            if not re.search("[A-Z][A-Z]+", concept):
                final_list.append(word.lower())
            else:
                final_list.append(word)
        resolved = " ".join(final_list)
#         print (resolved, "suggestion", response)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word.lower(), response["corrections"][word][0].lower())
                
        
        print (concept, resolved)
        inp_spellchecked.append(resolved)
    else:
        inp_spellchecked.append(concept)

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

# stopwords = ["has", "is", "a", "an", "the"]
stopwords = ["has"]
inp_stemmed = []
for elem in inp_spellchecked:
    words = " ".join([word for word in elem.split() if word not in stopwords])
    words = words.replace("-", " ")
    inp_stemmed.append(words)


embeds = np.array(extractUSEEmbeddings(inp_stemmed))
embeds = np.array([np.zeros(embeds.shape[1],)] + list(embeds))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))


emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}


error list index out of range
registation deadline registration deadline
viza viz
reviewes reviewed
memeber registration fee member registration fee
accpet if room rating accept if room rating
programme programmer
scientifically organised by scientifically organized by
sponzorship sponsorship
has programme has programmer
sponzor sponsor
author cd proceedings included author cd proceedings included
is a topis of conference parts is a tops of conference parts
site URL site URL
holded by holed by
accepts hardcopy submissions accepts hard copy submissions
organised by organized by
technic activity technical activity
logo URL logo URL
sponzor fee sponsor fee
was a committe co-chair of was a committee co-chair of
author attendee cd registration fee author attendee cd registration fee
is writen by is write by
cd proceening cd proceeding
has VAT has VAT
webmaster web master
technically organises technically organists
programme brochure programmer brochure
LCD projector LCD projector
computer n

In [38]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, prop=False):
    ont_obj = Ontology("conference_ontologies/" + ont + ".owl")
    triples = ont_obj.get_triples()
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append([e2])
            neighbours_dict[e2][2].append([e1])
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append([e2])
            neighbours_dict[e2][3].append([e1])
        elif d == "Subclass":
            neighbours_dict[e2][1].append([e1])
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].split(".")[0]

    
    for entity in neighbours_dict:
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue

    if prop:
        prop_triples = ont_obj.get_triples(subclass_of=False)
        neighbours_dict_props = {c: [[c], [], []] for a,b,c,d in prop_triples}
        for e1, e2, p, d in prop_triples:
            neighbours_dict_props[p][1].extend([e1])
#             else:
#                 ne1ighbours_dict_props[p][1].extend([e1])
#             if e2 not in datatypes:
            neighbours_dict_props[p][2].extend([e2])
#             else:
#                 neighbours_dict_props[p][2].extend([e2])
        neighbours_dict_props = {ont + "#" + p: [list(OrderedSet([ont + "#" + e for e in elem])) for elem in neighbours_dict_props[p]] for p in neighbours_dict_props}
        return neighbours_dict_props

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}

neighbours_dicts_prop = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts_prop = {**neighbours_dicts_prop, **get_one_hop_neighbours(ont, True)}

    

# max_neighbours = np.max(flatten([[len(el[e]) for e in el] for el in neighbours_dicts.values()]))

# neighbours_dicts_prop = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont, True) for ont in list(set(flatten(ontologies_in_alignment)))}
# max_neighbours_prop = np.max(flatten([[[len(elem) for elem in el[e]] for e in el] for el in neighbours_dicts_prop.values()]))

# max_neighbours = np.max([max_neighbours, max_neighbours_prop])
# neighbours_dicts = {ont: {key: neighbours_dicts[ont][key] + ["<UNK>" for i in range(max_neighbours -len(neighbours_dicts[ont][key]))]
#               for key in neighbours_dicts[ont]} for ont in neighbours_dicts}

# neighbours_dicts_prop = {ont: {key: [e + ["<UNK>" for i in range(max_neighbours -len(e))] for e in neighbours_dicts_prop[ont][key]]
#               for key in neighbours_dicts_prop[ont]} for ont in neighbours_dicts_prop}

# ontologies_in_alignment = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment]
emb_indexer, emb_indexer_inv, emb_vals = pickle.load(open("Input/data_conf.pkl", "rb"))[1:4]

# f = open("Input/data_conf_oaei_german_aml_prop_thresh.pkl", "wb")
# pickle.dump([data_ent, data_prop, aml_data_ent, aml_data_prop, data_german, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts, neighbours_dicts_prop, ontologies_in_alignment], f)



list index out of range
list index out of range


In [39]:
f = open("Input/data_conf_oaei_german_aml_prop_thresh.pkl", "rb")
data_ent, data_prop, aml_data_ent, aml_data_prop, data_german, emb_indexer, emb_indexer_inv, emb_vals, _, neighbours_dicts_prop, ontologies_in_alignment = pickle.load(f)
f.close()
f = open("Input/data_conf_oaei_german_aml_prop_thresh.pkl", "wb")
pickle.dump([data_ent, data_prop, aml_data_ent, aml_data_prop, data_german, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts, neighbours_dicts_prop, ontologies_in_alignment], f)



In [33]:
set(data_ent.values())

{0, '1.0'}

In [ ]:
### ontologies_in_alignment = pickle.load(open("data_path.pkl", "rb"))[-1]
ontologies_in_alignment

In [28]:
cos_sim(*extractUSEEmbeddings(['late paid applicant', 'late registered participant']))

0.6148052215576172

In [28]:
# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []
# all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
ontologies_in_alignment = [tuple(pair) for pair in ontologies_in_alignment]
for i in list(range(0, len(ontologies_in_alignment)-1, 3)):
    test_onto = ontologies_in_alignment[i+1:i+3]
    for ont_pair in test_onto:
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0] + "-" + ont_pair[1]
        java_command = "java -jar AML_v3.1/AgreementMakerLight.jar -s conference_ontologies/" + a + ".owl" + \
                            " -t conference_ontologies/" + b + ".owl -o AML-test-results/" + c + ".rdf -a"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    print (os.listdir("AML-test-results/"))
    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem]])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and is_test(test_onto, key)])
    fp = len([elem for elem in pred_aml if not data[elem]])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    _ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

['iasted-sigkdd.rdf', 'cmt-ekaw.rdf']
0.7916666666666666 0.7307692307692307 0.76 0.7421874999999999 0.7786885245901639
['conference-edas.rdf', 'cmt-sigkdd.rdf']
0.8181818181818182 0.6206896551724138 0.7058823529411765 0.6521739130434783 0.7692307692307693
['conference-sigkdd.rdf', 'conference-confOf.rdf']
0.7727272727272727 0.5666666666666667 0.6538461538461539 0.5985915492957746 0.7203389830508474
['edas-iasted.rdf', 'cmt-conference.rdf']
0.75 0.5294117647058824 0.6206896551724139 0.5625 0.6923076923076923
['edas-sigkdd.rdf', 'ekaw-iasted.rdf']
0.875 0.56 0.6829268292682927 0.603448275862069 0.7865168539325843
['cmt-confOf.rdf', 'edas-ekaw.rdf']
0.7916666666666666 0.48717948717948717 0.6031746031746031 0.5277777777777778 0.7037037037037036
['conference-ekaw.rdf', 'cmt-iasted.rdf']
0.7142857142857143 0.6896551724137931 0.7017543859649122 0.6944444444444444 0.7092198581560283
Final Results: [0.78764688 0.59776743 0.67546771 0.62587478 0.73714377]


In [29]:

# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []
prefix = "/data/Vivek/IBM/IBM-Internship/conference_ontologies/"
for i in list(range(0, len(ontologies_in_alignment)-1, 3)):
    test_onto = ontologies_in_alignment[i+1:i+3]
    tp_tot, fn_tot, fp_tot = [], [], []
    for ont_pair in test_onto:
        a, b, c = prefix + ont_pair[0], prefix + ont_pair[1], ont_pair[0] + "-" + ont_pair[1]
        !mkdir $c
        java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  a + ".owl file:" + b + ".owl " + \
                        "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        
        pred_aml = [l.strip().split("\t")[:2] for l in open(c + "/logmap2_mappings.tsv", "r").read().split("\n")[:-1]]
        pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
        tp = [elem for elem in pred_aml if data[elem]]
        fn = [key for key in gt_mappings if key not in set(pred_aml) and is_test([tuple(ont_pair)], key)]
        fp = [elem for elem in pred_aml if not data[elem]]
        
        tp_tot.extend(tp)
        fn_tot.extend(fn)
        fp_tot.extend(fp)
        
        !rm -rf $c
   
    precision = len(tp_tot)/(len(tp_tot)+len(fp_tot))
    recall = len(tp_tot)/(len(tp_tot)+len(fn_tot))
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    
    
print ("Final Results:", np.mean(results, axis=0))

0.76 0.7307692307692307 0.7450980392156863 0.7364341085271318 0.753968253968254
0.8461538461538461 0.7586206896551724 0.8 0.7746478873239436 0.8270676691729323
0.7857142857142857 0.7333333333333333 0.7586206896551724 0.7432432432432431 0.7746478873239436
0.75 0.4411764705882353 0.5555555555555556 0.4807692307692307 0.6578947368421052
0.7777777777777778 0.56 0.6511627906976745 0.5932203389830509 0.7216494845360826
0.7727272727272727 0.4358974358974359 0.5573770491803278 0.4775280898876404 0.6692913385826772
0.5925925925925926 0.5517241379310345 0.5714285714285714 0.5594405594405594 0.583941605839416
Final Results: [0.75499511 0.6016459  0.66274896 0.62361192 0.71263728]


In [13]:
import pickle
ontologies_in_alignment = pickle.load(open("data_multi_rootpath.pkl", "rb"))[-1]

In [30]:
print ("Best thresholds: {}".format([1,2,3]))

Best thresholds: [1, 2, 3]


In [20]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

(167, 1240)

In [23]:
ontologies_in_alignment

[['confOf', 'sigkdd'],
 ['iasted', 'sigkdd'],
 ['cmt', 'ekaw'],
 ['confOf', 'iasted'],
 ['conference', 'edas'],
 ['cmt', 'sigkdd'],
 ['ekaw', 'sigkdd'],
 ['conference', 'confOf'],
 ['conference', 'sigkdd'],
 ['confOf', 'edas'],
 ['cmt', 'conference'],
 ['edas', 'iasted'],
 ['conference', 'iasted'],
 ['edas', 'sigkdd'],
 ['ekaw', 'iasted'],
 ['cmt', 'edas'],
 ['edas', 'ekaw'],
 ['cmt', 'confOf'],
 ['confOf', 'ekaw'],
 ['conference', 'ekaw'],
 ['cmt', 'iasted']]

In [22]:
list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))

[('conference', 'iasted'),
 ('ekaw', 'sigkdd'),
 ('cmt', 'sigkdd'),
 ('edas', 'ekaw'),
 ('conference', 'sigkdd'),
 ('iasted', 'sigkdd'),
 ('confOf', 'sigkdd'),
 ('ekaw', 'iasted'),
 ('conference', 'ekaw'),
 ('cmt', 'ekaw'),
 ('edas', 'sigkdd'),
 ('confOf', 'iasted'),
 ('cmt', 'conference'),
 ('cmt', 'confOf'),
 ('cmt', 'iasted'),
 ('conference', 'edas'),
 ('confOf', 'edas'),
 ('edas', 'iasted'),
 ('confOf', 'ekaw'),
 ('conference', 'confOf'),
 ('cmt', 'edas')]

In [83]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




registeered applicant registered applicant
technically organised by technically organized by
ngo no
sponzorship sponsorship


In [31]:
cos_sim(*extractUSEEmbeddings(["die lebensmittel", "food"]))

0.24726325273513794

In [ ]:
for

In [78]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(4)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(4)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)
create_comparison_file("test_v2.pkl", 2)
create_comparison_file("test_resolved.pkl", 3)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

7796

In [14]:
ontologies_in_alignment = pickle.load(open("data_path.pkl", "rb"))[-1]
ontologies_in_alignment

[['confOf', 'sigkdd'],
 ['iasted', 'sigkdd'],
 ['cmt', 'ekaw'],
 ['confOf', 'iasted'],
 ['conference', 'edas'],
 ['cmt', 'sigkdd'],
 ['ekaw', 'sigkdd'],
 ['conference', 'confOf'],
 ['conference', 'sigkdd'],
 ['confOf', 'edas'],
 ['cmt', 'conference'],
 ['edas', 'iasted'],
 ['conference', 'iasted'],
 ['edas', 'sigkdd'],
 ['ekaw', 'iasted'],
 ['cmt', 'edas'],
 ['edas', 'ekaw'],
 ['cmt', 'confOf'],
 ['confOf', 'ekaw'],
 ['conference', 'ekaw'],
 ['cmt', 'iasted']]

In [72]:
d = {('confOf#Organization', 'sigkdd#Organizator'): (1,2,3,4),
 ('iasted#Document', 'sigkdd#Document'): (5,6,78,8)}
[[str(el) for el in flatten(el)] for el in d.items()]

[['confOf#Organization', 'sigkdd#Organizator', '1', '2', '3', '4'],
 ['iasted#Document', 'sigkdd#Document', '5', '6', '78', '8']]

In [34]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

left ('Chair_PC', 'Program_Chair') PC Program_Chair
left ('Chair_PC', 'Program_Committee') PC Program_Committee
left ('Chair_PC', 'Program_Committee_member') PCM Program_Committee
left ('Member_PC', 'Program_Chair') PC Program_Chair
left ('Member_PC', 'Program_Committee') PC Program_Committee
left ('Member_PC', 'Program_Committee_member') PCM Program_Committee
left ('Chair_PC', 'Presenter_city') PC Presenter_city
left ('Member_PC', 'Presenter_city') PC Presenter_city
left ('OC_Member', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Member', 'Organizing_Committee') OC Organizing_Committee
left ('PC_Member', 'Program_Chair') PC Program_Chair
left ('PC_Member', 'Program_Committee') PC Program_Committee
left ('PC_Member', 'Program_Committee_member') PCM Program_Committee
left ('OC_Chair', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Chair', 'Organizing_Committee') OC Organizing_Committee
left ('PC_Chair', 'Program_Chair') PC Program_Chair
left ('PC_C

In [48]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [27]:
cos_sim(*extractUSEEmbeddings(["Conference Banquet", "Dinner Banquet"]))

0.8085169792175293

In [26]:
fn

[(('confOf#hasEmail', 'sigkdd#E-mail'), 0.9161555776735063),
 (('confOf#Chair_PC', 'sigkdd#Program_Chair'), 0.8290806880788957),
 (('iasted#Student_registration_fee', 'sigkdd#Registration_Student'),
  0.9156892709934972),
 (('iasted#Deadline_for_notification_of_acceptance',
   'sigkdd#Deadline_Author_notification'),
  0.5085231269229767),
 (('iasted#Nonmember_registration_fee', 'sigkdd#Registration_Non-Member'),
  0.8098175068174337),
 (('cmt#ConferenceMember', 'ekaw#Conference_Participant'), 0.3532655254301476),
 (('cmt#Author', 'ekaw#Paper_Author'), 0.9288647440450568),
 (('cmt#writtenBy', 'ekaw#reviewWrittenBy'), 0.7592434303979944),
 (('cmt#hasBeenAssigned', 'ekaw#reviewerOfPaper'), 0.2502871547766108),
 (('cmt#assignedTo', 'ekaw#hasReviewer'), 0.3499445900379554),
 (('cmt#PaperFullVersion', 'ekaw#Regular_Paper'), 0.8379052493933772),
 (('confOf#Administrative_event', 'iasted#Activity_before_conference'),
  0.37740041094526244),
 (('conference#has_the_first_name', 'edas#hasFirstNam

In [53]:
scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)


In [61]:
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
    
inp_case_handled

['pay',
 'rejected by',
 'Registration SIGMOD Member',
 'is connected with',
 'NGO',
 'overhead projector',
 'name of conference',
 'call for participation',
 'coffee break',
 'scientifically organised by',
 'volunteer',
 'publisher',
 'regular',
 'add program committee member',
 'contact email',
 'part of event',
 'dinner banquet',
 'social program',
 'decision',
 'is sent by',
 'was a committee chair of',
 'organisation',
 'paper',
 'assign external reviewer',
 'was a member of',
 'has cost amount',
 'session chair',
 'single level conference',
 'hotel',
 'deadline hotel reservation',
 'country',
 'important dates',
 'paper',
 'has parts',
 'event',
 'paper due on',
 'is the 1th part of',
 'has surname',
 'wireless communications topic',
 'two level conference',
 'has first name',
 'bid',
 'is designed for',
 'author',
 'double hotel room',
 'review form',
 'presentation',
 'subject area',
 'positive integer',
 'deadline abstract submission',
 'has a degree',
 'is present in',
 'rela

In [56]:
data

NameError: name 'data' is not defined

In [67]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [81]:
# from transformers import XLNetTokenizer, XLNetModel
# import torch
# import scipy
# import torch.nn as nn
# import torch.nn.functional as F

# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

import time

t = time.time()
input_ids = torch.tensor(tokenizer.encode("fastigial nucleus", add_special_tokens=True)).unsqueeze(0)
outputs = model(input_ids)
last_hidden_states = outputs[0].mean(1)

print (t-time.time())
# input_ids = torch.tensor(tokenizer.encode("femur", add_special_tokens=True)).unsqueeze(0) 

# outputs1 = model(input_ids)
# last_hidden_states1 = outputs1[0].mean(1)

# cos = nn.CosineSimilarity(dim=1, eps=1e-6)
# cos(last_hidden_states, last_hidden_states1)

-0.14206647872924805


In [97]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
import logging
from datetime import datetime
import sys

model_name = "emilyalsentzer/Bio_ClinicalBERT"

word_embedding_model = models.Transformer(model_name)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=False,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=True)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


OSError: Can't load config for 'bert-large-nli-mean-tokens'. Make sure that:

- 'bert-large-nli-mean-tokens' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'bert-large-nli-mean-tokens' is the correct path to a directory containing a config.json file



In [101]:
model = SentenceTransformer('bert-large-nli-mean-tokens')

cos_sim(*model.encode(["My brother plays guitar", "The sun is shining"]))

0.2498054802417755

In [102]:
cos_sim(*extractUSEEmbeddings(["My brother plays guitar", "The sun is shining"]))

0.1406574696302414